In [2]:
import pandas as pd
import tensorflow_hub as hub
import tensorflow.compat.v1 as tfv1
import tensorflow as tfv2
tfv1.disable_eager_execution()
import sklearn 

In [3]:
## read data

In [4]:
## take all data
import pandas as pd
col_names = ["target", "ids", "date", "flag", "user", "text"]

data1 = pd.read_csv('./training.1600000.processed.noemoticon.csv',
            encoding = "ISO-8859-1",
            names=col_names)
data = pd.DataFrame(columns= ["sentiment","text"])
data.sentiment = data1.target
data.text = data1.text

In [5]:
# take part of data
col_names = ["target", "ids", "date", "flag", "user", "text"]

data1 = pd.read_csv('./training.1600000.processed.noemoticon.csv',
            encoding = "ISO-8859-1",
            names=col_names)

data_0 = data1[data1['target'] == 0].loc[:7800]
data_1 = data1[data1['target'] == 4].loc[800000:807800]
data_3 = pd.concat([data_0,data_1])


data = pd.DataFrame(columns= ["sentiment","text"])
data.sentiment = data_3.target
data.text = data_3.text
data = sklearn.utils.shuffle(data)

In [6]:
## remove @XXXXX #XXX number and links

In [7]:
import re
def remove(pattern,text):
    document = []
    for doc in text:
        tmp = re.sub(pattern, "", doc)
        document.append(tmp)
    return document

text = "-Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"



pattern1 = r'http[a-z]*:+\S+'
pattern2 = r'@[A-Za-z0-9_.]*'
pattern3 = r'#[A-Za-z0-9_.]*'
pattern4 = r'[0-9]'

#document = re.sub(pattern1, "", text)
#document = re.sub(pattern2, "", document)
#document = re.sub(pattern3, "", document)
#document = re.sub(pattern4, "", document)


document = remove(pattern1, data.text)
document = remove(pattern2, document)
document = remove(pattern3, document)
document = remove(pattern4, document)


#view = "#"
#check = []
#for text in document:
#    check.append(view in text)

##get label
label = []
for doc in data.sentiment:
    label.append(doc)
## get color list for PCA plot
color = []
for tmp in label:
    if tmp == "neutral":
        color.append("Blue")
    elif tmp == "positive":
        color.append("Red")
    elif tmp == "negative":
        color.append("Green")

## tokenize
import tokenizers.pre_tokenizers
tokenizers = tokenizers.pre_tokenizers.Sequence([tokenizers.pre_tokenizers.Punctuation(),tokenizers.pre_tokenizers.Whitespace()])
document_tokenize = [] 
for sentence in document:
    tokens = tokenizers.pre_tokenize_str(sentence)
    tmp = []
    for word in tokens:
        tmp.append(word[0])
    document_tokenize.append(tmp)

## remove all empty
check = []
for i in range(0,len(document_tokenize)):
    if len(document_tokenize[i]) == 0 :
        check.append(i)

for i in range(0,len(check)):
    index = check[i] - i
    del (document[index])
    del (document_tokenize[index])
    del (label[index])    
    
## save token_length   
import pickle
test = open("token_length.pickle","wb")
pickle.dump(token_length, test)
test.close()    

max_num  = max(len(sen) for sen in document_tokenize)
document_tokenize_padding = []
for sequence in document_tokenize:
    tmp = sequence
    for i in range(0,max_num-len(tmp)):
        tmp.append("")
    document_tokenize_padding.append(tmp)


NameError: name 'token_length' is not defined

In [ ]:
##load elmo
elmo = hub.Module("https://tfhub.dev/google/elmo/3",trainable=True)

In [ ]:
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:

sequence_input = document_tokenize_padding
token_length = [len(sen) for sen in document_tokenize]
inputs = {"tokens": sequence_input,"sequence_len":token_length}

def elmo_to_vectors(sequence_input,layer,types):
    embeddings = elmo( inputs = sequence_input, signature="tokens", as_dict=True)[layer]
    init1 = tfv1.global_variables_initializer()
    init2 = tfv1.tables_initializer()
    sess = tfv1.Session()
    sess.run(init1)
    sess.run(init2)
    
    if types == "maxout":
        maxout = tfv2.math.reduce_max(embeddings,axis=1)
        maxout = sess.run(maxout)
        return maxout
    elif types == "meanout":
        meanout = tfv2.math.reduce_mean(embeddings,axis=1)
        meanout = sess.run(meanout)
        return meanout
    else:
        embeddings_vectors = sess.run(embeddings)
        return embeddings_vectors
          

    return embeddings_vectors,euclidean_norm,logsumexp,maxout,meanout
def iteration_to_vect(inputs,layer,number,types):
    results = []

    for i in range(0,len(inputs['sequence_len']),number):
        each = len(inputs['sequence_len']) // 100+1
        percent = i // each 
        print("[","-"*percent," "*(100-percent),"]",percent,"%","(",i,")")
        inputs_tmp = {"tokens":inputs['tokens'][i:i+number],"sequence_len":inputs['sequence_len'][i:i+number]}
        results_tmp = elmo_to_vectors(inputs_tmp,layer,types)
        results = results + results_tmp.tolist()

    return results


In [ ]:
del data1
del document_tokenize
del document
import gc 
gc.collect()

In [ ]:
## get elmo layer

In [ ]:
layer = "elmo"
types = "embeddings_vectors"
embeddings_vectors_elmo = iteration_to_vect(inputs,layer,100,types)

In [ ]:
types = "maxout"
maxout_elmo  = iteration_to_vect(inputs,layer,100,types)

In [ ]:
types = "meanout"
meanout_emo = iteration_to_vect(inputs,layer,100,types)

In [ ]:
import pickle

test = open("label.pickle","wb")
pickle.dump(label, test)
test.close()

test = open("embeddings_vectors_elmo.pickle","wb")
pickle.dump(embeddings_vectors_elmo, test)
test.close()

test = open("maxout_elmo.pickle","wb")
pickle.dump(maxout_elmo, test)
test.close()

In [ ]:
test = open("meanout_emo.pickle","wb")
pickle.dump(meanout_emo, test)
test.close()

In [ ]:
layer = "word_emb"
types = "embeddings_vectors"
embeddings_vectors_word_emb = iteration_to_vect(inputs,layer,100,types)
types = "maxout"
maxout_word_emb = iteration_to_vect(inputs,layer,100,types)
types = "meanout"
meanout_word_emb = iteration_to_vect(inputs,layer,100,types)

In [ ]:
test = open("embeddings_vectors_word_emb.pickle","wb")
pickle.dump(embeddings_vectors_word_emb, test)
test.close()

test = open("maxout_word_emb.pickle","wb")
pickle.dump(maxout_word_emb, test)
test.close()


In [ ]:

test = open("meanout_word_emb.pickle","wb")
pickle.dump(meanout_word_emb, test)
test.close()


In [ ]:
layer = "lstm_outputs1"   
types = "embeddings_vectors"
embeddings_vectors_lstm_outputs1 = iteration_to_vect(inputs,layer,100,types)
types = "maxout"
maxout_lstm_outputs1 = iteration_to_vect(inputs,layer,100,types)
types = "meanout"
meanout_lstm_outputs1 = iteration_to_vect(inputs,layer,100,types)


In [ ]:
test = open("embeddings_vectors_lstm_outputs1.pickle","wb")
pickle.dump(embeddings_vectors_lstm_outputs1, test)
test.close()

test = open("maxout_lstm_outputs1.pickle","wb")
pickle.dump(maxout_word_emb, test)
test.close()

test = open("meanout_lstm_outputs1.pickle","wb")
pickle.dump(meanout_lstm_outputs1, test)
test.close()

In [ ]:
layer = "lstm_outputs2"   
types = "embeddings_vectors"
embeddings_vectors_lstm_outputs2 = iteration_to_vect(inputs,layer,100,types)
types = "maxout"
maxout_lstm_outputs2 = iteration_to_vect(inputs,layer,100,types)
types = "meanout"
meanout_lstm_outputs2 = iteration_to_vect(inputs,layer,100,types)

In [ ]:
test = open("lstm_outputs2.pickle","wb")
pickle.dump(lstm_outputs2, test)
test.close()

test = open("lstm_outputs2.pickle","wb")
pickle.dump(lstm_outputs2, test)
test.close()

test = open("lstm_outputs2.pickle","wb")
pickle.dump(lstm_outputs2, test)
test.close()